## Documentation

To read more about the index API, visit the [docs](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-index_.html).

![index_api_docs](../images/index_api_docs.png)

## Connect to ElasticSearch

In [4]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch('http://localhost:9200')
client_info = es.info()
print('Connected to Elasticsearch!')
pprint(client_info.body)

Connected to Elasticsearch!
{'cluster_name': 'es-docker-cluster',
 'cluster_uuid': '68vsKryIR7Ss49bLh7mz5Q',
 'name': 'es01',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-12-16T10:09:08.849001802Z',
             'build_flavor': 'default',
             'build_hash': 'd8972a71dbbd64ff17f2f4dba9ca2c3fe09fb100',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '10.3.2',
             'minimum_index_compatibility_version': '8.0.0',
             'minimum_wire_compatibility_version': '8.19.0',
             'number': '9.2.3'}}


## Insert one document

Create a dummy index just to test inserting one document

In [5]:
es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(index='my_index')

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

In [6]:
document = {
    'title': 'title',
    'text': 'text',
    'created_on': '2024-09-22',
}
response = es.index(index='my_index', body=document)
response

ObjectApiResponse({'_index': 'my_index', '_id': '5D3EaZsBX61qEH3b8Eib', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

In [ ]:
# Add a document to my_index
POST /my_index/_doc
{
    "id": "park_rocky-mountain",
    "title": "Rocky Mountain",
    "description": "Bisected north to south by the Continental Divide, this portion of the Rockies has ecosystems varying from over 150 riparian lakes to montane and subalpine forests to treeless alpine tundra."
}

> Note: The mapping is created automatically by the elasticsearch, if not specified at the time index creation. Generally the object mapping is same as the inserted object.
    

> Note: The added documents are not checked for uniquness, to make it unique, set the unique identifier has _id, by

```sh
// Indexing Request (using the SKU as the _id)
POST /products/_create/ABC-123
{
  "product_sku": "ABC-123",
  "name": "Super Widget",
  "price": 19.99
}
```

The `response` object contains the result of the operation. If we successfully inserted the document, then `result = created`. Each document has an `id` and is fragmented into `shards`.

In [7]:
print(response["result"])

created


In [13]:
print(response["_shards"])

{'total': 2, 'successful': 1, 'failed': 0}


In [14]:
print(response["_id"])

5D3EaZsBX61qEH3b8Eib


In [15]:
print(response["_index"])

my_index


## Insert multiple documents

Just do the same step but in a for loop

In [16]:
import json

dummy_data = json.load(open("../data/dummy_data.json"))
dummy_data

[{'title': 'Sample Title 1',
  'text': 'This is the first sample document text.',
  'created_on': '2024-09-22'},
 {'title': 'Sample Title 2',
  'text': 'Here is another example of a document.',
  'created_on': '2024-09-24'},
 {'title': 'Sample Title 3',
  'text': 'The content of the third document goes here.',
  'created_on': '2024-09-24'}]

In [19]:
def insert_document(document):
    response = es.index(index='my_index', body=document)
    return response


def print_info(response):
    print(f"""Document ID: {response['_id']} is '{
          response["result"]}' and is split into {response['_shards']['total']} shards.""")


for document in dummy_data:
    response = insert_document(document)
    print_info(response)

Document ID: 7T3QaZsBX61qEH3b90iE is 'created' and is split into 3 shards.
Document ID: 7j3QaZsBX61qEH3b90jH is 'created' and is split into 3 shards.
Document ID: 7z3QaZsBX61qEH3b90jV is 'created' and is split into 3 shards.


## Print mapping

In [18]:
from pprint import pprint

index_mapping = es.indices.get_mapping(index='my_index')
pprint(index_mapping["my_index"]["mappings"]["properties"])

{'created_on': {'type': 'date'},
 'description': {'fields': {'keyword': {'ignore_above': 256,
                                        'type': 'keyword'}},
                 'type': 'text'},
 'id': {'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}},
        'type': 'text'},
 'text': {'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}},
          'type': 'text'},
 'title': {'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}},
           'type': 'text'}}


## Manual mapping

In [20]:
es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(index='my_index')

mapping = {
    'properties': {
        'created_on': {'type': 'date'},
        'text': {
            'type': 'text',
            'fields': {
                'keyword': {
                    'type': 'keyword',
                    'ignore_above': 256
                }
            }
        },
        'title': {
            'type': 'text',
            'fields': {
                'keyword': {
                    'type': 'keyword',
                    'ignore_above': 256
                }
            }
        }
    }
}

es.indices.put_mapping(index='my_index', body=mapping)

index_mapping = es.indices.get_mapping(index='my_index')
pprint(index_mapping["my_index"]["mappings"]["properties"])

{'created_on': {'type': 'date'},
 'text': {'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}},
          'type': 'text'},
 'title': {'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}},
           'type': 'text'}}


In [ ]:
PUT /my_index/
{
  "mappings": {
    "properties": {
      "created_on": {
        "type": "date"
      },
      "text": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      },
      "title": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      }
    }
  }
}

In [21]:
mapping = {
    'properties': {
        'created_on': {'type': 'date'},
        'text': {
            'type': 'text',
            'fields': {
                'keyword': {
                    'type': 'keyword',
                    'ignore_above': 256
                }
            }
        },
        'title': {
            'type': 'text',
            'fields': {
                'keyword': {
                    'type': 'keyword',
                    'ignore_above': 256
                }
            }
        }
    }
}

es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(index='my_index', mappings=mapping)

index_mapping = es.indices.get_mapping(index='my_index')
pprint(index_mapping["my_index"]["mappings"]["properties"])

{'created_on': {'type': 'date'},
 'text': {'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}},
          'type': 'text'},
 'title': {'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}},
           'type': 'text'}}


> Note: The mapping is not strictly enforced, the set mapping can change if differently mapped item is added to the index.